In [1]:
import re

import kagglehub
import keras
import keras_hub
import keras_tuner
import pandas as pd
import pkuseg
import tensorflow as tf
from datasets import Dataset
from pypinyin import lazy_pinyin
from keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Input,
    StringLookup,
    TextVectorization,
)
from keras.models import Model


path = kagglehub.dataset_download("noxmoon/chinese-official-daily-news-since-2016")

print("Path to dataset files:", path)

Path to dataset files: /Users/zhongjie/.cache/kagglehub/datasets/noxmoon/chinese-official-daily-news-since-2016/versions/1


# Création du corpus

In [2]:
dataset = pd.read_csv(path+"/chinese_news.csv")
# Print dataset information
print("Dataset information:")
print(dataset.info())
# Print dataset head
print("Dataset head:")
print(dataset.head())

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20738 entries, 0 to 20737
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      20738 non-null  object
 1   tag       20738 non-null  object
 2   headline  20738 non-null  object
 3   content   20631 non-null  object
dtypes: object(4)
memory usage: 648.2+ KB
None
Dataset head:
         date   tag                                           headline  \
0  2016-01-01  详细全文  陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...   
1  2016-01-01  详细全文                             中央军委印发《关于深化国防和军队改革的意见》   
2  2016-01-01  详细全文                           《习近平关于严明党的纪律和规矩论述摘编》出版发行   
3  2016-01-01  详细全文                                 以实际行动向党中央看齐 向高标准努力   
4  2016-01-01  详细全文                                 【年终特稿】关键之年 改革挺进深水区   

                                             content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...  
1  经中央军委主席习近平批准，中央军委近日印发了《关

In [3]:
# Prétraitement de content (suppression des caractères non chinois, normalisation des espaces)
def clean_content(text):
    if not isinstance(text, str):
        return ""
    
    # Garder les caractères chinois et ponctuation chinoise
    text = re.sub(r"[^\u4e00-\u9fff\u3000-\u303F\uff00-\uffef]", "", text)
    
    # Normaliser les espaces (rare, mais au cas où)
    text = text.replace(" ", "").strip()

    return text

# Remplacer les valeurs manquantes par une chaîne vide
dataset["content"] = dataset["content"].fillna("")

# Appliquer le prétraitement à la colonne 'content'
dataset['cleaned_content'] = dataset['content'].apply(clean_content)

# Filtrer les lignes où 'cleaned_content' est vide
dataset = dataset[dataset["cleaned_content"].str.strip() != ""].reset_index(drop=True)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after preprocessing:")
print(dataset[['content', 'cleaned_content']].head())

seg = pkuseg.pkuseg()
dataset["tokens"] = dataset["cleaned_content"].apply(lambda x: seg.cut(x))

# Aperçu
print(dataset["tokens"].head())

Dataset after preprocessing:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                                     cleaned_content  
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会年月日在...  
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。《意见》强调，...  
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...  
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...  
4  刚刚过去的年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，集中发力...  
0    [中国, 人民, 解放军, 陆军, 领导, 机构, 、, 中国, 人民, 解放军, 火箭军,...
1    [经, 中央军委, 主席, 习近平, 批准, ，, 中央军委, 近日, 印发, 了, 《, ...
2    [由, 中共中央, 纪律, 检查, 委员会, 、, 中共中央, 文献, 研究室, 编辑, 的...
3    [广大, 党员, 干部, 正在, 积极, 学习, 习近平, 总书记, 在, 中央, 政治局,...
4    [刚刚, 过去, 的, 年, ，, 是, 全面, 深化, 改革, 的, 关键, 之, 年, ...
Name: tokens, dtype: object


In [4]:
# convert the content column to pinyin
t9_map = {
    "@": "1", ".": "1", ":": "1",
    "a": "2", "b": "2", "c": "2",
    "d": "3", "e": "3", "f": "3",
    "g": "4", "h": "4", "i": "4",
    "j": "5", "k": "5", "l": "5",
    "m": "6", "n": "6", "o": "6",
    "p": "7", "q": "7", "r": "7", "s": "7",
    "t": "8", "u": "8", "v": "8",
    "w": "9", "x": "9", "y": "9", "z": "9",
    "1": "1", "2": "2", "3": "3", "4": "4",
    "5": "5", "6": "6", "7": "7", "8": "8",
    "9": "9", "0": "0", " ": "0",
    "。":"。", "，":"，", "？":"？", "！":"！",
}

# Fonction pour convertir une chaîne de caractères en code T9
def pinyin_to_t9(text):
    t9_code = ""
    if pd.isna(text):
        return ""
    for char in text.lower():
        t9_code += t9_map.get(char, char)  # Conserver les caractères non mappés
    return t9_code

def validate_t9(t9_code):
    # Vérifie que le code T9 est numérique (ou vide pour ponctuation)
    return bool(re.match(r'^[0-9]+$', t9_code)) or t9_code in {"。", "，", "？", "！"}

def generer_sequence_contextuelle(row):
    tokens = row["tokens"]
    sequence = []
    for token in tokens:
        if not isinstance(token, str) or not re.search(r'[\u4e00-\u9fff]', token):
            continue
        for char, py in zip(token, lazy_pinyin(token)):
            t9 = pinyin_to_t9(py)
            if validate_t9(t9):  # Vérifier que le T9 est valide
                sequence.append(f"{char}|{py}|{t9}")
    return ' '.join(sequence)

dataset["char_pinyin_t9_sequence"] = dataset.apply(generer_sequence_contextuelle, axis=1)

# Filtrer les lignes où 'char_pinyin_t9_sequence' est vide
dataset = dataset[dataset["char_pinyin_t9_sequence"].str.strip() != ""].reset_index(drop=True)

# Sauvegarder le fichier
dataset[["char_pinyin_t9_sequence"]].to_csv("sequences_char_pinyin_t9.csv", index=False)

# Afficher les 5 premières lignes du DataFrame après le prétraitement
print("Dataset after generating sequences:")
print(dataset[['content', 'char_pinyin_t9_sequence']].head())

Dataset after generating sequences:
                                             content  \
0  中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...   
1  经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...   
2  由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...   
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...   
4  刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...   

                             char_pinyin_t9_sequence  
0  中|zhong|94664 国|guo|486 人|ren|736 民|min|646 解|...  
1  经|jing|5464 中|zhong|94664 央|yang|9264 军|jun|58...  
2  由|you|968 中|zhong|94664 共|gong|4664 中|zhong|94...  
3  广|guang|48264 大|da|32 党|dang|3264 员|yuan|9826 ...  
4  刚|gang|4264 刚|gang|4264 过|guo|486 去|qu|78 的|de...  


# Création du dataset pour le modèle

In [5]:
# Transformer en séquences complètes
input_t9_sequences = []
target_char_sequences = []
MAX_SEQUENCE_LENGTH = 100

for seq in dataset["char_pinyin_t9_sequence"]:
    triplets = seq.strip().split(" ")
    t9_seq = []
    char_seq = []
    
    # Extraire les paires char|T9 pour chaque phrase
    for triplet in triplets[:MAX_SEQUENCE_LENGTH]:  # Tronquer à MAX_SEQUENCE_LENGTH
        parts = triplet.split("|")
        if len(parts) == 3:
            char, _, t9 = parts
            if validate_t9(t9):  # Vérifier que le T9 est valide
                char_seq.append(char)
                t9_seq.append(t9)
    
    # Ajouter les séquences T9 et caractères si non vides
    if t9_seq and char_seq:
        input_t9_sequences.append(" ".join(t9_seq))
        target_char_sequences.append("".join(char_seq)) # A voir si on garde les espaces ou pas

# Créer un DataFrame
df_sequences = pd.DataFrame({
    "input_t9_sequence": input_t9_sequences,
    "target_char_sequence": target_char_sequences
})

# Filtrer les séquences vides (par précaution)
df_sequences = df_sequences[df_sequences["input_t9_sequence"].str.strip() != ""]
df_sequences = df_sequences[df_sequences["target_char_sequence"].str.strip() != ""]

In [6]:
print("DataFrame sequences:")
print(df_sequences.head())

DataFrame sequences:
                                   input_t9_sequence  \
0  94664 486 736 646 543 3264 586 58 586 5464 326...   
1  5464 94664 9264 586 934 948 94 94 546 7464 74 ...   
2  968 94664 4664 94664 9264 54 58 5426 242 934 9...   
3  48264 32 3264 9826 426 28 94364 924 54 54 983 ...   
4  4264 4264 486 78 33 6426 744 7826 6426 7436 48...   

                                target_char_sequence  
0  中国人民解放军陆军领导机构中国人民解放军火箭军中国人民解放军战略支援部队成立大会年月日在八一...  
1  经中央军委主席习近平批准中央军委近日印发了关于深化国防和军队改革的意见意见强调党的十八大以来...  
2  由中共中央纪律检查委员会中共中央文献研究室编辑的习近平关于严明党的纪律和规矩论述摘编一书近日...  
3  广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话大家纷纷表示要把践...  
4  刚刚过去的年是全面深化改革的关键之年改革集中发力在制约经济社会发展的深层次矛盾集中发力在妨碍...  


In [7]:
# Utiliser tf.data.Dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((df_sequences['input_t9_sequence'], df_sequences['target_char_sequence'])).prefetch(tf.data.AUTOTUNE)
tf_dataset.take(1).get_single_element()

(<tf.Tensor: shape=(), dtype=string, numpy=b'94664 486 736 646 543 3264 586 58 586 5464 326 54 468 94664 486 736 646 543 3264 586 486 5426 586 94664 486 736 646 543 3264 586 9426 583 944 9826 28 384 24364 54 32 484 6426 983 74 924 22 94 32 568 5664 94664 58 9464 94664 4664 94664 9264 9664 748 54 486 542 948 94 94664 9264 586 934 948 94 94 546 7464 94264 58 586 486 5426 586 9426 583 944 9826 28 384 7468 98 586 74 2464 944 986 24 324 2426 3264 94664 9264 43 94664 9264'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'\xe4\xb8\xad\xe5\x9b\xbd\xe4\xba\xba\xe6\xb0\x91\xe8\xa7\xa3\xe6\x94\xbe\xe5\x86\x9b\xe9\x99\x86\xe5\x86\x9b\xe9\xa2\x86\xe5\xaf\xbc\xe6\x9c\xba\xe6\x9e\x84\xe4\xb8\xad\xe5\x9b\xbd\xe4\xba\xba\xe6\xb0\x91\xe8\xa7\xa3\xe6\x94\xbe\xe5\x86\x9b\xe7\x81\xab\xe7\xae\xad\xe5\x86\x9b\xe4\xb8\xad\xe5\x9b\xbd\xe4\xba\xba\xe6\xb0\x91\xe8\xa7\xa3\xe6\x94\xbe\xe5\x86\x9b\xe6\x88\x98\xe7\x95\xa5\xe6\x94\xaf\xe6\x8f\xb4\xe9\x83\xa8\xe9\x98\x9f\xe6\x88\x90\xe7\xab\x8b\xe5\xa4\xa7\xe4\xbc\x9a\

## Encoder les données pour Keras

In [8]:
# TextVectorization
input_tv = keras.layers.TextVectorization(output_mode='int',
                                          split='character',
                                          standardize=None,
                                          ragged=True,)

target_tv = keras.layers.TextVectorization(output_mode='int',
                                           split='character',
                                           standardize=None,
                                           ragged=True,)
tmp_t9_ds = tf_dataset.map(lambda t9, target: tf.strings.reduce_join(tf.strings.split(t9, " "), separator=""))
t9_ds = tf_dataset.map(lambda t9, target: t9)
target_ds = tf_dataset.map(lambda t9, target: target)
input_tv.adapt(tmp_t9_ds)
target_tv.adapt(target_ds)

2025-04-11 15:26:11.665197: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-04-11 15:26:43.022125: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
# Le vocabulaire pour t9 input
print(input_tv.get_vocabulary())

['', '[UNK]', np.str_('4'), np.str_('6'), np.str_('2'), np.str_('3'), np.str_('8'), np.str_('9'), np.str_('5'), np.str_('7'), np.str_('。')]


In [55]:
@tf.function
def split_and_vectorize(t9_seq, char_seq, context_size=5):
    # Split chaque séquence T9 (par espace)
    t9_parts = tf.strings.split(t9_seq, " ")  # tf.TensorShape([None])

    # Split la séquence de caractères (par caractère UTF-8)
    char_parts = tf.strings.unicode_split(char_seq, "UTF-8")

    # Padder la séquence de caractères pour avoir le contexte
    char_seq_padded = tf.strings.join([tf.constant("     "), char_seq])
    char_parts_padded = tf.strings.unicode_split(char_seq_padded, "UTF-8")

    # Vérification : même longueur (sécurité)
    assert_op = tf.debugging.assert_equal(tf.shape(t9_parts)[0], tf.shape(char_parts)[0])

    with tf.control_dependencies([assert_op]):
        
        # Vectorisation : chaque chiffre dans chaque t9_part est un caractère
        vectorized_t9 = input_tv(t9_parts)       # RaggedTensor: [nb_sous_seq, longueur_t9]
        vectorized_target = target_tv(char_parts)  # RaggedTensor: [nb_sous_seq, 1] ou int32

        # Convertit RaggedTensor en Tensor avec padding (0 par défaut, qui sera ignoré lors du Embedding Layer)
        t9_tensor = vectorized_t9.to_tensor(default_value=0)
        target_tensor = vectorized_target.to_tensor(default_value=0)
        target_tensor = tf.squeeze(target_tensor, axis=-1)
        
        # Contexte
        vectorized_target_padded = target_tv(char_parts_padded).to_tensor(default_value=0)
        padded_seq_len = tf.shape(vectorized_target_padded)[0]
        contexts = tf.TensorArray(dtype=tf.int64, size=padded_seq_len - context_size)
        for i in tf.range(padded_seq_len - context_size):
            context = vectorized_target_padded[i:i+context_size]
            contexts = contexts.write(i, context)
        contexts_tensor = contexts.stack()
        contexts_tensor = tf.squeeze(contexts_tensor, axis=-1)

        return {"t9_input": t9_tensor, "context_input": contexts_tensor}, target_tensor

transformed_dataset = tf_dataset.map(
    lambda t9, target: split_and_vectorize(t9, target),
    num_parallel_calls=20
)

In [96]:
transformed_dataset.take(1).get_single_element()

({'t9_input': <tf.Tensor: shape=(100, 5), dtype=int64, numpy=
  array([[7, 2, 3, 3, 2],
         [2, 6, 3, 0, 0],
         [9, 5, 3, 0, 0],
         [3, 2, 3, 0, 0],
         [8, 2, 5, 0, 0],
         [5, 4, 3, 2, 0],
         [8, 6, 3, 0, 0],
         [8, 6, 0, 0, 0],
         [8, 6, 3, 0, 0],
         [8, 2, 3, 2, 0],
         [5, 4, 3, 0, 0],
         [8, 2, 0, 0, 0],
         [2, 3, 6, 0, 0],
         [7, 2, 3, 3, 2],
         [2, 6, 3, 0, 0],
         [9, 5, 3, 0, 0],
         [3, 2, 3, 0, 0],
         [8, 2, 5, 0, 0],
         [5, 4, 3, 2, 0],
         [8, 6, 3, 0, 0],
         [2, 6, 3, 0, 0],
         [8, 2, 4, 3, 0],
         [8, 6, 3, 0, 0],
         [7, 2, 3, 3, 2],
         [2, 6, 3, 0, 0],
         [9, 5, 3, 0, 0],
         [3, 2, 3, 0, 0],
         [8, 2, 5, 0, 0],
         [5, 4, 3, 2, 0],
         [8, 6, 3, 0, 0],
         [7, 2, 4, 3, 0],
         [8, 6, 5, 0, 0],
         [7, 2, 2, 0, 0],
         [7, 6, 4, 3, 0],
         [4, 6, 0, 0, 0],
         [5, 6, 2, 0, 0],
  

## Split train-valid-test

In [57]:
c = transformed_dataset.reduce(0, lambda x,_:x+1).numpy()

shuffled_ds = transformed_dataset.shuffle(buffer_size=c, seed=42)

train_size = c * 80 // 100
test_size = c * 10 // 100
val_size = c - train_size - test_size

ds_train = shuffled_ds.take(train_size).prefetch(tf.data.AUTOTUNE)
ds_val = shuffled_ds.skip(train_size).take(val_size).prefetch(tf.data.AUTOTUNE)
ds_test = shuffled_ds.skip(train_size+val_size).take(test_size).prefetch(tf.data.AUTOTUNE)

print("Taille du train :", ds_train.cardinality().numpy())
print("Taille du validation :", ds_val.cardinality().numpy())
print("Taille du test :", ds_test.cardinality().numpy())

Taille du train : 16503
Taille du validation : 2064
Taille du test : 2062


In [98]:
ds_train.take(1).get_single_element()

({'t9_input': <tf.Tensor: shape=(75, 5), dtype=int64, numpy=
  array([[8, 2, 3, 0, 0],
         [3, 2, 4, 3, 0],
         [7, 3, 0, 0, 0],
         [2, 6, 3, 0, 0],
         [7, 2, 4, 0, 0],
         [8, 2, 4, 3, 2],
         [7, 2, 6, 0, 0],
         [4, 2, 4, 3, 0],
         [9, 6, 0, 0, 0],
         [4, 2, 3, 2, 0],
         [4, 2, 3, 3, 2],
         [2, 4, 2, 0, 0],
         [9, 2, 4, 3, 0],
         [7, 2, 3, 3, 2],
         [5, 4, 0, 0, 0],
         [9, 2, 5, 3, 2],
         [3, 3, 3, 2, 0],
         [7, 5, 0, 0, 0],
         [4, 6, 0, 0, 0],
         [8, 2, 4, 3, 0],
         [4, 5, 0, 0, 0],
         [7, 2, 4, 3, 0],
         [9, 2, 2, 0, 0],
         [8, 2, 5, 0, 0],
         [7, 2, 2, 0, 0],
         [3, 6, 0, 0, 0],
         [9, 2, 4, 3, 0],
         [7, 2, 4, 3, 0],
         [3, 4, 2, 0, 0],
         [4, 2, 3, 2, 0],
         [4, 2, 3, 3, 2],
         [2, 4, 2, 0, 0],
         [8, 5, 2, 0, 0],
         [8, 2, 0, 0, 0],
         [5, 4, 0, 0, 0],
         [9, 2, 5, 3, 2],
   

# Modèle

Sogou T9 est une méthode d’entrée intelligente qui :

- Prend des séquences numériques (ex. : "94664 486" pour "zhong guo").
- Génère des séquences de caractères chinois (ex. : "中国").
- Utilise le contexte (mots précédents) pour désambiguïser les prédictions.
- Est optimisé pour la vitesse et la précision, souvent avec des modèles entraînés sur de vastes corpus.

Pour reproduire cela, il faut utiliser un modèle seq2seq avec un encodeur-décodeur (2 entrées) :

- Encodeur : Lit la séquence T9 et la compresse en une représentation contextuelle.
- Décodeur : Génère la séquence de caractères chinois à partir de cette représentation.

[Functional API](https://keras.io/guides/functional_api/)

In [107]:
ds_train_padded = ds_train.padded_batch(
    128,
    padded_shapes=(
        {
            "t9_input": [100, 5],
            "context_input": [100, 5]
        },
        [100]  # Pour les étiquettes
    ),
    padding_values=(
        {
            "t9_input": tf.constant(0, dtype=tf.int64),
            "context_input": tf.constant(0, dtype=tf.int64)
        },
        tf.constant(0, dtype=tf.int64)  # Pour les étiquettes
    ),
    drop_remainder=True,
).prefetch(tf.data.AUTOTUNE)

ds_val_padded = ds_val.padded_batch(
    128,
    padded_shapes=(
        {
            "t9_input": [100, 5],
            "context_input": [100, 5]
        },
        [100]  # Pour les étiquettes
    ),
    padding_values=(
        {
            "t9_input": tf.constant(0, dtype=tf.int64),
            "context_input": tf.constant(0, dtype=tf.int64)
        },
        tf.constant(0, dtype=tf.int64)  # Pour les étiquettes
    ),
    drop_remainder=True,
).prefetch(tf.data.AUTOTUNE)

ds_test_padded = ds_test.padded_batch(
    128,
    padded_shapes=(
        {
            "t9_input": [100, 5],
            "context_input": [100, 5]
        },
        [100]  # Pour les étiquettes
    ),
    padding_values=(
        {
            "t9_input": tf.constant(0, dtype=tf.int64),
            "context_input": tf.constant(0, dtype=tf.int64)
        },
        tf.constant(0, dtype=tf.int64)  # Pour les étiquettes
    ),
    drop_remainder=True,
).prefetch(tf.data.AUTOTUNE)

In [110]:
ds_train_padded.take(1)

<_TakeDataset element_spec=({'t9_input': TensorSpec(shape=(128, 100, 5), dtype=tf.int64, name=None), 'context_input': TensorSpec(shape=(128, 100, 5), dtype=tf.int64, name=None)}, TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

In [111]:
ds_train_padded.take(1).get_single_element()

DataLossError: {{function_node __wrapped__DatasetToSingleElement_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Attempted to pad to a smaller size than the input element. [Op:DatasetToSingleElement] name: 

# Génération